In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#import os

#zip_path = '/content/drive/MyDrive/kaggle_dataset/synthetic-faces-high-quality-sfhq-part-1.zip'  # Ganti 'namafile.zip' sesuai nama file kamu
#print(os.path.exists(zip_path))


In [ ]:
#!unzip '/content/drive/MyDrive/kaggle_dataset/synthetic-faces-high-quality-sfhq-part-1.zip' -d '/content/drive/MyDrive/kaggle_dataset/unzipped'


In [ ]:
#import os

# Path folder hasil unzip kamu
#input_dir = '/content/drive/MyDrive/kaggle_dataset/unzipped/images/images'


# Ambil semua path file gambar (jpg, png)
#image_paths = [os.path.join(dp, f)
#               for dp, dn, filenames in os.walk(input_dir)
#               for f in filenames if f.lower().endswith(('.jpg', '.png'))]

#print(f"Total gambar ditemukan: {len(image_paths)}")

In [1]:
!pip uninstall -y numpy mediapipe
!pip install numpy==1.24.4 mediapipe==0.10.14

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 7.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: jax
    Found exist

In [2]:
import kagglehub


# Download latest version
path = kagglehub.dataset_download("selfishgene/synthetic-faces-high-quality-sfhq-part-2")

print("Path to dataset files:", path)

100%|██████████| 13.5G/13.5G [02:22<00:00, 102MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4


In [3]:
import os

print(os.listdir(path))

['landmarks', 'pretrained_features', 'images', 'segmentations', 'a small sample (650 images)', 'a tiny sample (140 images)']


In [4]:
# Cek path sebenarnya
full_image_folder = os.path.join(path, "images")
print("Path ke folder gambar:", full_image_folder)

# Lihat isi folder
files = os.listdir(full_image_folder)
print(f"Jumlah file: {len(files)}")
print("Contoh file:", files[:10])

Path ke folder gambar: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4/images
Jumlah file: 1
Contoh file: ['images']


In [5]:
print("Isi folder:", os.listdir(full_image_folder))
print("Path lengkap:", full_image_folder)

Isi folder: ['images']
Path lengkap: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4/images


In [6]:
import random

image_paths = []
for root, dirs, files in os.walk(full_image_folder):
    for file in files:
        if file.lower().endswith(('.jpg', '.png')):
            image_paths.append(os.path.join(root, file))

# Sampling
sample_size = 50000
sampled_image_paths = random.sample(image_paths, min(sample_size, len(image_paths)))

print(f"Jumlah sampel gambar yang diambil: {len(sampled_image_paths)}")

Jumlah sampel gambar yang diambil: 50000


In [7]:
import cv2
import numpy as np
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh

def get_face_skin_tone(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            h, w, _ = image.shape
            face_landmarks = results.multi_face_landmarks[0]

            # Titik pipi kiri dan kanan
            indices = [234, 454]
            colors = []
            for idx in indices:
                x = int(face_landmarks.landmark[idx].x * w)
                y = int(face_landmarks.landmark[idx].y * h)
                if 0 <= x < w and 0 <= y < h:
                    colors.append(image[y, x])
            if colors:
                avg_color = np.mean(colors, axis=0)  # BGR
                return avg_color
        return None

In [8]:
def classify_season_from_color(bgr):
    # LAB untuk brightness
    bgr_lab = np.uint8([[bgr]])
    lab = cv2.cvtColor(bgr_lab, cv2.COLOR_BGR2LAB)[0][0]
    L = lab[0]  # Lightness

    # HSV untuk hue & saturation
    bgr_hsv = cv2.cvtColor(np.uint8([[bgr]]), cv2.COLOR_BGR2HSV)[0][0]
    H, S, V = bgr_hsv

    # Deteksi undertone (warm vs cool)
    if 10 < H < 35:
        undertone = 'warm'
    elif H < 10 or H > 160:
        undertone = 'cool'
    else:
        undertone = 'neutral'

    # Gabungan brightness dan undertone
    if undertone == 'warm':
        return 'Spring' if L >= 140 else 'Autumn'
    elif undertone == 'cool':
        return 'Winter' if L >= 140 else 'Summer'
    else:
        return 'Winter' if L >= 140 else 'Autumn'

In [9]:
output_base = '/content/drive/MyDrive/labeled_faces'
season_labels = ['autumn', 'winter', 'summer', 'spring']

for label in season_labels:
    os.makedirs(os.path.join(output_base, label), exist_ok=True)

In [10]:
import shutil

labels = []

# ✅ Ganti ke sampled_image_paths
print(f"Total gambar yang akan diproses: {len(sampled_image_paths)}")

for idx, path in enumerate(sampled_image_paths):
    try:
        print(f"\n[{idx+1}/{len(sampled_image_paths)}] Memproses: {path}")

        color = get_face_skin_tone(path)
        if color is not None:
            print(f"  > Warna kulit terdeteksi (BGR): {color}")

            season = classify_season_from_color(color)
            print(f"  > Dikelompokkan ke musim: {season}")

            labels.append((path, season))

            # Siapkan nama file dan path tujuan
            filename = os.path.basename(path)
            dest_path = os.path.join(output_base, season.lower(), filename)

            if not os.path.exists(dest_path):  # Hindari overwrite
                shutil.move(path, dest_path)
                print(f"  > Dipindahkan ke: {dest_path}")
            else:
                print("  > File sudah ada, dilewati.")
        else:

             print("  > Tidak terdeteksi wajah pada gambar ini.")

    except Exception as e:
        print(f"[ERROR] Gagal memproses {path}: {e}")


Total gambar yang akan diproses: 50000

[1/50000] Memproses: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4/images/images/SFHQ_pt2_00038645.jpg


/usr/local/lib/python3.11/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Output streaming akan dipotong hingga 5000 baris terakhir.

[49001/50000] Memproses: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4/images/images/SFHQ_pt2_00087094.jpg
  > Warna kulit terdeteksi (BGR): [ 55.  70. 101.]
  > Dikelompokkan ke musim: Autumn
  > Dipindahkan ke: /content/drive/MyDrive/labeled_faces/autumn/SFHQ_pt2_00087094.jpg

[49002/50000] Memproses: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4/images/images/SFHQ_pt2_00088080.jpg
  > Warna kulit terdeteksi (BGR): [138.  140.5 175. ]
  > Dikelompokkan ke musim: Winter
  > Dipindahkan ke: /content/drive/MyDrive/labeled_faces/winter/SFHQ_pt2_00088080.jpg

[49003/50000] Memproses: /root/.cache/kagglehub/datasets/selfishgene/synthetic-faces-high-quality-sfhq-part-2/versions/4/images/images/SFHQ_pt2_00077776.jpg
  > Warna kulit terdeteksi (BGR): [ 83.   92.  115.5]
  > Dikelompokkan ke musim: Summer
  > Dipindahkan ke: /content/dr

In [11]:
import pandas as pd

df = pd.DataFrame(labels, columns=['image_path', 'season_label'])
df.to_csv('/content/drive/MyDrive/labeled_faces_labels.csv', index=False)

In [12]:
import os

for label in ['spring', 'summer', 'autumn', 'winter']:
    folder = os.path.join(output_base, label)
    count = len([f for f in os.listdir(folder) if f.endswith(('.jpg', '.png'))])
    print(f"{label.capitalize()}: {count} file")

Spring: 3414 file
Summer: 24602 file
Autumn: 17173 file
Winter: 4811 file


In [14]:
# Salin dataset dari Drive ke local runtime
src_path = "/content/drive/MyDrive/labeled_faces"
dst_path = "/content/labeled_faces"

if not os.path.exists(dst_path):
    shutil.copytree(src_path, dst_path)
    print("✅ Dataset berhasil disalin ke local runtime.")
else:
    print("📁 Folder sudah tersedia di local runtime.")

✅ Dataset berhasil disalin ke local runtime.


In [15]:
import os
import shutil

base_dir = '/content/labeled_faces'  # Asumsikan dataset sudah disalin ke local
output_zips = {}

# Loop tiap kelas dan buat zip-nya
for label in ['spring', 'summer', 'autumn', 'winter']:
    folder_path = os.path.join(base_dir, label)
    zip_path = f'/content/{label}_faces.zip'

    # Zip folder kelas
    shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)
    output_zips[label] = zip_path

print("✅ Semua folder berhasil dikompres ke ZIP.")


✅ Semua folder berhasil dikompres ke ZIP.


In [ ]:
from google.colab import files

# Unduh satu per satu
for label, zip_path in output_zips.items():
    print(f"⬇️ Mengunduh: {zip_path}")
    files.download(zip_path)